# Jass Rule Based Player

### by Ruben Nunez & Jordan Suter 
Import Statements

In [3]:
from jass.game.game_util import *
from jass.game.game_sim import GameSim
from jass.game.game_observation import GameObservation
from jass.game.const import *
from jass.game.rule_schieber import RuleSchieber
from jass.agents.agent import Agent
from jass.agents.agent_random_schieber import AgentRandomSchieber
from jass.arena.arena import Arena

In [5]:
# This distributes the cards randomly among the 4 players.
hands = deal_random_hand()
print(hands.shape)

(4, 36)


In [12]:
# There is an entry for each player, to access the cards of the first player
cards = hands[0,:]
print(cards)

[0 0 0 0 0 1 0 0 0 0 0 0 0 0 1 0 1 0 0 0 0 0 1 0 0 0 1 0 1 0 1 1 0 0 0 1]


In [13]:
# This should be 9 cards
assert(cards.sum() == 9)

# The cards can be converted to other formats for easier reading or processing
print(convert_one_hot_encoded_cards_to_str_encoded_list(cards))

# Each card is encoded as a value between 0 and 35.
print(convert_one_hot_encoded_cards_to_int_encoded_list(cards))

['D9', 'H9', 'H7', 'S10', 'S6', 'CK', 'CJ', 'C10', 'C6']
[5, 14, 16, 22, 26, 28, 30, 31, 35]


In [14]:
# There is a method to count colors too
colors = count_colors(cards)
print(colors)

[1 2 2 4]


Another possibility to select trump is by assigning a value to each card, depending on whether the color is trump or not. This table is from the Maturawork of Daniel Graf from 2009: "Jassen auf Basis der Spieltheorie".

In [15]:
# Score for each card of a color from Ace to 6

# score if the color is trump
trump_score = [15, 10, 7, 25, 6, 19, 5, 5, 5]
# score if the color is not trump
no_trump_score = [9, 7, 5, 2, 1, 0, 0, 0, 0]
# score if obenabe is selected (all colors)
# obenabe_score = [14, 10, 8, 7, 5, 0, 5, 0, 0,]
# score if uneufe is selected (all colors)
# uneufe_score = [0, 2, 1, 1, 5, 5, 7, 9, 11]

## Our rule based agent

In [44]:
class RuleBasedAgent(Agent):
    def __init__(self):
        super().__init__()
        # we need a rule object to determine the valid cards
        self._rule = RuleSchieber()
        
        
    def calculate_trump_selection_score(self, cards, trump: int) -> int:
        score = 0;
        for card in cards:
            suit = int(card / 9)
            exact_card = card % 9
            # print("card=" + str(card) + "; suit=" + str(suit) + "; exact=" + str(exact_card))
            score += trump_score[exact_card] if trump == suit else no_trump_score[exact_card]

        return score
        

    def action_trump(self, obs: GameObservation) -> int:
        """
        Determine trump action for the given observation
        Args:
            obs: the game observation, it must be in a state for trump selection

        Returns:
            selected trump as encoded in jass.game.const or jass.game.const.PUSH
        """
        # add your code here using the function above
        card_list = convert_one_hot_encoded_cards_to_int_encoded_list(obs.hand)
        threshold = 68
        scores = [0,0,0,0]
        for suit in range(0,4):
            trump_score = self.calculate_trump_selection_score(card_list, suit)
            scores[suit] = trump_score
        
        best_score = max(scores)
        best_suit = scores.index(best_score)
            
        if best_score <= threshold and obs.player < 1:
            return PUSH
        else:
            return best_suit
        

    def action_play_card(self, obs: GameObservation) -> int:
        """
        Determine the card to play.

        Args:
            obs: the game observation

        Returns:
            the card to play, int encoded as defined in jass.game.const
        """
        ## TODO Hier müssen wir den vor programmierten Entscheidungsbaum einfügen statt rnd 
        
        valid_cards = self._rule.get_valid_cards_from_obs(obs)
        
        
        # we use the global random number generator here
        # np.random.choice(np.flatnonzero(valid_cards))
#         print(valid_cards)
        print(convert_one_hot_encoded_cards_to_str_encoded_list(valid_cards))
        return np.flatnonzero(valid_cards)[0]
    
    

## Arena Test

In [61]:
arena = Arena(nr_games_to_play=1)
arena.set_players(RuleBasedAgent(), AgentRandomSchieber(), RuleBasedAgent(), AgentRandomSchieber())
arena.play_all_games()

['S6']
['SJ', 'S9', 'S8']
['DJ', 'D6', 'HQ', 'HJ', 'H9', 'H8', 'CK', 'C7']
['DA', 'DQ']
['CK', 'C7']
['C10', 'C9', 'C6']
['HQ', 'HJ', 'H9', 'H8']
['HK']
['D6', 'HJ', 'H9', 'H8', 'C7']
['S9', 'S8']
['DQ', 'S8', 'C9', 'C6']
['HJ', 'H9', 'H8', 'C7']
['C9', 'C6']
['C7']
['H9', 'H8']
['S8', 'C6']
['H8']
['C6']



In [43]:
print(arena.points_team_0.sum(), arena.points_team_1.sum())

157.0 0.0
